# --- `Day 8`: Title ---

In [1]:
import aocd
import re
import operator
from collections import Counter, defaultdict, deque
from itertools import combinations
from functools import reduce, lru_cache

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

def count(iterable, predicate = bool):
    return sum([1 for item in iterable if predicate(item)])

def first(iterable, default = None):
    return next(iter(iterable), default)

def lmap(func, *iterables):
    return list(map(func, *iterables))

def ints(s):
    return lmap(int, re.findall(r"-?\d+", s))

def words(s):
    return re.findall(r"[a-zA-Z]+", s)

def list_diff(x):
    return [b - a for a, b in zip(x, x[1:])]

def binary_to_int(lst):
    return int("".join(str(i) for i in lst), 2)

def get_column(lst, index):
    return [x[index] for x in lst]

In [3]:
def parse_line(line): 
    return str(line)
    
def parse_input(input):
    return list(map(parse_line, input.splitlines()))

In [4]:
final_input = parse_input(aocd.get_data(day=8, year=2021))
print(final_input[:5])

['bgeacd dbfag bcadegf agdce dgfbce bgc bdgca aedcgf bc abec | gcdfbe cbea bc gbc', 'bdeag gdbaec cd dgc abcfg ebcd dgfabe cdfeag cgadb bdagfce | becd acfgde bgcaed eadgbc', 'acbfg bcf ebacg fb fcbgea cbdfge cgeabd agcfd aebf bdaefgc | fbcdeg cfb ebgca bf', 'bfcde dfgb gecbdf fdc fdcega fd ebdca gfcaedb gcfbe cbagef | dbfec gbfd bcdfe dfc', 'dbgeaf bad acdfbe ab agbdf bgcdf fdgae fcgbdea fgecda egab | ba dab abd gbae']


In [46]:
test_input = parse_input('''\
be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
''')

print(test_input)

['be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe', 'edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc', 'fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg']


## Solution 1

In [52]:
def solve_1(input):
    count = 0
    for line in input:
        a, b = line.split(" | ")
        data = words(a)
        output = words(b)
        for i in data:
            x = Counter(i)
            
        output = words(b)
        for z in output:
            y = Counter(z)
            l = len(y)
            if l == 2 or l == 4 or l == 3 or l == 7:
                count += 1
    return count

solve_1(test_input)

8

In [53]:
f"Solution 1: {solve_1(final_input)}"

'Solution 1: 352'

## Solution 2

In [50]:
def solve_2(input):
    count = 0
    for line in input:
        digits = ""
        a, b = line.split(" | ")
        data = words(a)
        output = words(b)
        
        numbers = []
        results = defaultdict(set)
        for i in data:
            x = Counter(i)
            l = len(x)
            signals = set(x.keys())
            numbers.append(signals)
            if l == 2:
                results[1] = signals
            elif l == 4:
                results[4] = signals
            elif l == 3:
                results[7] = signals
            elif l == 7:
                results[8] = signals
        
        # set 3
        for i in numbers:
            if len(i) == 5:
                z = i.intersection(results[1])
                if len(z) == 2:
                    results[3] = i
        
        # set 9
        for i in numbers:
            if len(i) == 6:
                z = i.intersection(results[3])
                if len(z) == 5:
                    results[9] = i
        # set 0
        for i in numbers:
            if len(i) == 6:
                if i != results[9]:
                    z = i.intersection(results[1])
                    if len(z) == 2:
                        results[0] = i
                    
        # set 6
        for i in numbers:
            if len(i) == 6:
                if i != results[0] and i != results[9]:
                    results[6] = i
                    
        # set 5
        for i in numbers:
            if len(i) == 5:
                if i != results[3]:
                    z = i.intersection(results[6])
                    if len(z) == 5:
                        results[5] = i
                
        for z in output:
            y = Counter(z)
            h = set(y.keys())
            
            if h == results[0]:
                digits += '0'
            elif h == results[1]:
                digits += '1'
            elif h == results[3]:
                digits += '3'
            elif h == results[4]:
                digits += '4'
            elif h == results[5]:
                digits += '5'
            elif h == results[6]:
                digits += '6'
            elif h == results[7]:
                digits += '7'
            elif h == results[8]:
                digits += '8'
            elif h == results[9]:
                digits += '9'
            else:
                digits += '2'
        count += int(digits)
    return count
    
solve_2(test_input)

19372

In [51]:
f"Solution 2: {solve_2(final_input)}"

'Solution 2: 936117'